In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import cv2
import os

# Giả sử mô hình đã được huấn luyện và lưu
model = tf.keras.models.load_model('C:/Users/Admin/Desktop/CV/traffic_sign_cnn01.h5')  # Tải mô hình đã huấn luyện

# Danh sách nhãn lớp
class_names = [
    "Toc do toi da 20km/h", "Toc do toi da 30km/h", "Toc do toi da 50km/h", 
    "Toc do toi da 60km/h", "Toc do toi da 70km/h", "Toc do toi da 80km/h", 
    "Het gioi han toc do", "Toc do toi da 100km/h", "Toc do toi da 120km/h", 
    "Cam vuot", "Cam vuot xe tai", "Uu tien tai giao lo", "Duong uu tien", 
    "Nhuong duong", "Dung lai", "Cam xe qua", "Cam xe tai qua", 
    "Cam vao", "Canh bao chung", "Khuc cua nguy hiem ben trai", 
    "Khuc cua nguy hiem ben phai", "Duong cong doi", "Duong go ghe", 
    "Duong tron truot", "Duong hep ben phai", "Cong truong", 
    "Den giao thong phia truoc", "Nguoi di bo", "Tre em qua duong", 
    "Xe dap qua duong", "Can than bang tuyet", "Dong vat hoang da qua duong", 
    "Het moi gioi han", "Re phai phia truoc", "Re trai phia truoc", 
    "Chi di thang", "Chi re phai", "Chi re trai", "Giu ben phai", 
    "Giu ben trai", "Vong xuyen bat buoc", "Het cam vuot", 
    "Het cam vuot xe tai"
]
# Hàm dự đoán trên khung hình
def predict_frame(frame, model):
    img = cv2.resize(frame, (32, 32))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img, verbose=0)
    label = class_names[np.argmax(pred)]
    confidence = np.max(pred) * 100

    cv2.putText(frame, f"{label} ({confidence:.2f}%)", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    return frame

# Mở webcam
cap = cv2.VideoCapture(0)  # 0 là webcam mặc định

if not cap.isOpened():
    print("Không thể mở webcam.")
else:
    print("Đang nhận diện từ webcam... Nhấn 'q' để thoát.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Chuyển sang RGB
        frame = predict_frame(frame, model)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Chuyển lại BGR để hiển thị

        cv2.imshow('Traffic Sign Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Nhấn 'q' để thoát
            break

    cap.release()
    cv2.destroyAllWindows()

Đang nhận diện từ webcam... Nhấn 'q' để thoát.


In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk

# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model('C:/Users/Admin/Desktop/CV/traffic_sign_cnn01.h5')

# Danh sách nhãn lớp
class_names = [
    "Toc do toi da 20km/h", "Toc do toi da 30km/h", "Toc do toi da 50km/h", 
    "Toc do toi da 60km/h", "Toc do toi da 70km/h", "Toc do toi da 80km/h", 
    "Het gioi han toc do", "Toc do toi da 100km/h", "Toc do toi da 120km/h", 
    "Cam vuot", "Cam vuot xe tai", "Uu tien tai giao lo", "Duong uu tien", 
    "Nhuong duong", "Dung lai", "Cam xe qua", "Cam xe tai qua", 
    "Cam vao", "Canh bao chung", "Khuc cua nguy hiem ben trai", 
    "Khuc cua nguy hiem ben phai", "Duong cong doi", "Duong go ghe", 
    "Duong tron truot", "Duong hep ben phai", "Cong truong", 
    "Den giao thong phia truoc", "Nguoi di bo", "Tre em qua duong", 
    "Xe dap qua duong", "Can than bang tuyet", "Dong vat hoang da qua duong", 
    "Het moi gioi han", "Re phai phia truoc", "Re trai phia truoc", 
    "Chi di thang", "Chi re phai", "Chi re trai", "Giu ben phai", 
    "Giu ben trai", "Vong xuyen bat buoc", "Het cam vuot", 
    "Het cam vuot xe tai"
]

CONFIDENCE_THRESHOLD = 50  # Ngưỡng xác suất tối thiểu

def predict_frame(frame):
    img = cv2.resize(frame, (32, 32))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img, verbose=0)
    max_prob = np.max(pred) * 100
    label = class_names[np.argmax(pred)]
    
    if max_prob > CONFIDENCE_THRESHOLD:
        return f"{label} ({max_prob:.2f}%)"
    return None

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    detected_signs = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = predict_frame(frame)
        
        if result and result not in detected_signs:
            detected_signs.append(result)

    cap.release()
    return detected_signs

def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi"), ("Image files", "*.jpg;*.png")])
    if file_path:
        if file_path.endswith(('.mp4', '.avi')):
            detected_signs = process_video(file_path)
            result_list.delete(*result_list.get_children())
            for sign in detected_signs:
                result_list.insert("", "end", values=(sign,))
        else:
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = predict_frame(img)
            result_label.config(text=result if result else "Không phát hiện biển báo.")

def open_webcam():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        result_label.config(text="Không thể mở webcam.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = predict_frame(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if result:
            cv2.putText(frame, result, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('Traffic Sign Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Giao diện GUI
root = tk.Tk()
root.title("Traffic Sign Recognition")
root.geometry("600x500")
root.configure(bg="#f0f0f0")

tk.Label(root, text="Chọn phương thức nhận diện:", font=("Arial", 14), bg="#f0f0f0").pack(pady=10)

btn_open = tk.Button(root, text="Chọn ảnh/Video", command=open_file, font=("Arial", 12), bg="#007BFF", fg="white", padx=10, pady=5)
btn_open.pack(pady=5)

btn_webcam = tk.Button(root, text="Mở Webcam", command=open_webcam, font=("Arial", 12), bg="#28A745", fg="white", padx=10, pady=5)
btn_webcam.pack(pady=5)

result_label = tk.Label(root, text="", font=("Arial", 12), fg="blue", bg="#f0f0f0")
result_label.pack(pady=10)

# Bảng hiển thị kết quả
tk.Label(root, text="Danh sách biển báo nhận diện được:", font=("Arial", 12), bg="#f0f0f0").pack()

frame_table = tk.Frame(root)
frame_table.pack(pady=5)

columns = ("Biển báo",)  # Thêm dấu phẩy để xác định tuple
result_list = ttk.Treeview(frame_table, columns=columns, show="headings")
result_list.heading("Biển báo", text="Biển báo")


root.mainloop()